In [65]:
#libraries
import numpy as np 
import pandas as pd

**Вариант 2. Задание 6.**

\begin{equation*}
 \begin{cases}
    \frac{\partial u}{\partial t} = \frac{\partial}{r \partial r}(ru\frac{\partial u}{\partial r}) + \frac{\partial}{r^2 \partial \varphi}(u\frac{\partial u}{\partial \varphi}) , 0 < t,r \le 1, 0 < \varphi < \pi / 2 \\
    u(0, r, \varphi) = \frac{r^2cos^2(\varphi)}{7}, 0 \le r \le 1, 0 \le \varphi \le \pi / 2 \\
    u(t. 0, \varphi) = 0, 0 < t \le 1, 0 \le \varphi \le \pi / 2\\
    u(t, 1, \varphi) = \frac{cos^2(\varphi)}{(7 - 6t)}, 0 < t \le 1, 0 \le \varphi < \pi / 2 \\
    u(t, r, 0) = \frac{r^2}{(7 - 6t)}, 0 < r < 1, 0 < t \le 1 \\
    u(t, r, \pi / 2) = 0, 0 < r < 1, 0 < t \le 1    
 \end{cases}
\end{equation*}

In [66]:
def f_private(t, r, phi):
    '''
    функция частного решения
    '''
    return r**2 * (np.cos(phi))**2 / (7 - 6 * t)

def private_solution(t, r, phi):
    '''
    частное решение на сетке
    '''
    u = np.zeros((len(phi), len(r)))
    for i in range(len(phi)):
        u[i, :] = f_private(t, r, phi[i])
    return u

In [67]:
def grid(L, M, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, np.pi /2, M), np.linspace(0, 1, N)#менять первый и второй параметр, если изменятся границы

def sweep_mthd(a, b, c, d, m, l, u_ex):
    '''
    Метод прогонки
    '''
    alpha = np.zeros((m, l))
    beta = np.zeros((m, l))
    beta[1:-1, 0] = u_ex[1:-1,0]
    x = u_ex.copy()
    
    for i in range(1, l - 1):
        alpha[1:-1,i] = (-a(i) / (c(i) * alpha[1:-1,i-1] + b(i)))
        beta[1:-1,i] = (d(i) - c(i) * beta[1:-1,i-1]) / (b(i) + c(i) * alpha[1:-1,i-1])
    
    for i in reversed(range(1, l-1)):
        x[1:-1,i] = alpha[1:-1,i] * x[1:-1,i + 1]  + beta[1:-1,i]
    return x

def accuracy(u1, u2):
    '''
    Вычислят точность
    Returns bool
    '''
    max = 0
    epsilon = 0.01#точность
    for m in range(u1.shape[1]):
        for l in range(u1.shape[0]):
            if u2[m][l] != 0:
                num = abs((abs(u2[m][l]) - abs(u1[m][l])) / u2[m][l])
                if num - max > 0:
                    max = num
            if max - epsilon > 0:
                return False
    return True

def first_step(u, u1, r, m, n, mu, tau, hr):
    
    def a_l(l):
        return -hr * (l + 0.5) * (u1[1:-1, l + 1]**mu + u1[1:-1, l]**mu) * tau / (2 * r[l] * hr**2)
    def c_l(l):
        return -hr * (l - 0.5) * (u1[1:-1, l]**mu + u1[1:-1, l - 1]**mu) * tau / (2 * r[l] * hr**2)
    def b_l(l):    
        return 1 - a_l(l) - c_l(l)
    def d_l(l):
        return u[n, 1:- 1, l]
    l = len(r)
    u_p = sweep_mthd(a_l, b_l, c_l, d_l, m, l, u[n+1,:,:])

    return u_p

def second_step(u, u_tilda, u_f, r, m, mu, tau, hf):
    l = len(r)
    u1 = u
    u_tilda1 = u_tilda.T

    def a_m(m):
        return -(u1[1:-1, m + 1]**mu + u1[1:-1, m]**mu) * tau / (2 * (r[1:-1] * hf)**2)
    def c_m(m):
        return -(u1[1:-1, m]**mu + u1[1:-1, m - 1]**mu) * tau / (2 * (r[1:-1] * hf)**2)
    def b_m(m):
        return 1 - a_m(m) - c_m(m)
    def d_m(m):
        return u_tilda1[1:-1, m]
    
    return sweep_mthd(a_m, b_m, c_m, d_m, l, m, u_f.T).T

    
    
def numerical_solve(r, phi, t, mu):
    hr = 1 / (len(r) - 1)
    hf = np.pi / 2 / (len(phi) - 1)
    tau = 1 / (len(t) - 1)
    
    m = len(phi)
    
    u = np.zeros((len(t), len(phi), len(r)))
    
    for i in range(len(phi)):#первое граничное условие
        u[0, i, :] = f_private(0, r, phi[i])
    
    for i in range(1, len(t)):
        u[i, :, -1] = f_private(t[i], r[-1], phi)#третье граничное условие
        u[i, 0, 1:-1] = f_private(t[i], r[1:-1], phi[0])#четвертое граничное условие
   
    for i in range(len(t) - 1):
        u_smth = u[i].copy()
        
        while True:
            u_tilda = first_step(u, u_smth, r, m, i, mu, tau, hr)
            u_end = second_step(u_smth, u_tilda, u[i+1], r, m, mu, tau, hf)
            if accuracy(u_smth, u_end):
                u[i + 1] = u_end
                break
            else:
                u_smth = u_end
             
    return u

In [68]:
#ТО ШО МЕНЯТЬ
L = M = 11
N = 21
mu = 1
T = 1

#находим аналитическое решение
r_rep, phi_rep, _ = grid(L, M, N) 

u_private = private_solution(T, r_rep, phi_rep)


#находим численное рашение
r, phi, t = grid(L, M, N)
u_numeric = numerical_solve(r, phi, t, mu)

    
print('Max error:', np.max(np.abs(u_private - u_numeric[-1]))/2)



Max error: 0.06403889631326143


In [69]:
print('_____Analytical____')
f = pd.DataFrame(u_private, columns=r_rep, index=phi_rep)
f

_____Analytical____


,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0.000000,0.0,1.000000e-02,4.000000e-02,9.000000e-02,1.600000e-01,2.500000e-01,3.600000e-01,4.900000e-01,6.400000e-01,8.100000e-01,1.000000e+00
0.157080,0.0,9.755283e-03,3.902113e-02,8.779754e-02,1.560845e-01,2.438821e-01,3.511902e-01,4.780088e-01,6.243381e-01,7.901779e-01,9.755283e-01
0.314159,0.0,9.045085e-03,3.618034e-02,8.140576e-02,1.447214e-01,2.261271e-01,3.256231e-01,4.432092e-01,5.788854e-01,7.326519e-01,9.045085e-01
0.471239,0.0,7.938926e-03,3.175571e-02,7.145034e-02,1.270228e-01,1.984732e-01,2.858013e-01,3.890074e-01,5.080913e-01,6.430530e-01,7.938926e-01
0.628319,0.0,6.545085e-03,2.618034e-02,5.890576e-02,1.047214e-01,1.636271e-01,2.356231e-01,3.207092e-01,4.188854e-01,5.301519e-01,6.545085e-01
0.785398,0.0,5.000000e-03,2.000000e-02,4.500000e-02,8.000000e-02,1.250000e-01,1.800000e-01,2.450000e-01,3.200000e-01,4.050000e-01,5.000000e-01
0.942478,0.0,3.454915e-03,1.381966e-02,3.109424e-02,5.527864e-02,8.637288e-02,1.243769e-01,1.692908e-01,2.211146e-01,2.798481e-01,3.454915e-01
1.099557,0.0,2.061074e-03,8.244295e-03,1.854966e-02,3.297718e-02,5.152684e-02,7.419865e-02,1.009926e-01,1.319087e-01,1.669470e-01,2.061074e-01
1.256637,0.0,9.549150e-04,3.819660e-03,8.594235e-03,1.527864e-02,2.387288e-02,3.437694e-02,4.679084e-02,6.111456e-02,7.734812e-02,9.549150e-02
1.413717,0.0,2.447174e-04,9.788697e-04,2.202457e-03,3.915479e-03,6.117935e-03,8.809827e-03,1.199115e-02,1.566191e-02,1.982211e-02,2.447174e-02


In [70]:
print('_____Numerical____')
f1 = pd.DataFrame(u_numeric[-1], columns=r_rep, index=phi_rep)
f1


_____Numerical____


,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0.000000,0.0,0.010000,0.040000,0.090000,0.160000,0.250000,0.360000,0.490000,0.640000,0.810000,1.000000e+00
0.157080,0.0,0.006669,0.071099,0.176254,0.275161,0.371960,0.474275,0.585082,0.705368,0.835479,9.755283e-01
0.314159,0.0,0.003681,0.038339,0.112458,0.208985,0.310162,0.413685,0.523219,0.641000,0.767966,9.045085e-01
0.471239,0.0,0.002476,0.023422,0.068117,0.138830,0.229140,0.327648,0.431826,0.543517,0.664013,7.938926e-01
0.628319,0.0,0.001779,0.015606,0.042083,0.084543,0.148935,0.231659,0.324336,0.425123,0.534990,6.545085e-01
0.785398,0.0,0.001288,0.010783,0.026992,0.050106,0.085926,0.141539,0.214952,0.299705,0.394629,5.000000e-01
0.942478,0.0,0.000915,0.007430,0.017625,0.030126,0.046782,0.073616,0.119029,0.182111,0.257899,3.454915e-01
1.099557,0.0,0.000617,0.004919,0.011255,0.018113,0.025462,0.034882,0.052126,0.087023,0.139608,2.061074e-01
1.256637,0.0,0.000375,0.002946,0.006582,0.010183,0.013378,0.016208,0.019627,0.028182,0.053040,9.549150e-02
1.413717,0.0,0.000172,0.001341,0.002952,0.004456,0.005621,0.006336,0.006634,0.006803,0.008957,2.447174e-02


In [71]:
print('_____Errors____')
fe =  pd.DataFrame(np.abs(u_private - u_numeric[-1])/2, columns=r_rep, index=phi_rep)
fe

_____Errors____


,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
0.157080,0.0,1.543031e-03,1.603903e-02,4.422809e-02,5.953840e-02,6.403890e-02,6.154227e-02,5.353648e-02,4.051507e-02,2.265079e-02,0.0
0.314159,0.0,2.682145e-03,1.079438e-03,1.552593e-02,3.213169e-02,4.201751e-02,4.403104e-02,4.000474e-02,3.105714e-02,1.765687e-02,0.0
0.471239,0.0,2.731509e-03,4.166896e-03,1.666481e-03,5.903372e-03,1.533335e-02,2.092313e-02,2.140939e-02,1.771296e-02,1.047982e-02,0.0
0.628319,0.0,2.383147e-03,5.287317e-03,8.411389e-03,1.008922e-02,7.346222e-03,1.982207e-03,1.813651e-03,3.118671e-03,2.418967e-03,0.0
0.785398,0.0,1.855796e-03,4.608518e-03,9.004120e-03,1.494721e-02,1.953717e-02,1.923063e-02,1.502418e-02,1.014754e-02,5.185500e-03,0.0
0.942478,0.0,1.270196e-03,3.194822e-03,6.734448e-03,1.257648e-02,1.979535e-02,2.538036e-02,2.513083e-02,1.950192e-02,1.097460e-02,0.0
1.099557,0.0,7.218331e-04,1.662739e-03,3.647097e-03,7.432045e-03,1.303250e-02,1.965837e-02,2.443323e-02,2.244300e-02,1.366956e-02,0.0
1.256637,0.0,2.901263e-04,4.368223e-04,1.005871e-03,2.547636e-03,5.247520e-03,9.084682e-03,1.358193e-02,1.646620e-02,1.215421e-02,0.0
1.413717,0.0,3.632562e-05,1.812953e-04,3.747148e-04,2.701264e-04,2.486276e-04,1.236735e-03,2.678440e-03,4.429511e-03,5.432797e-03,0.0
